In [1]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()

True

# Firebase Init

In [2]:
service_account_info = {
    "type": os.getenv("FIREBASE_TYPE"),
    "project_id": os.getenv("FIREBASE_PROJECT_ID"),
    "private_key_id": os.getenv("FIREBASE_PRIVATE_KEY_ID"),
    "private_key": os.getenv("FIREBASE_PRIVATE_KEY"),
    "client_email": os.getenv("FIREBASE_CLIENT_EMAIL"),
    "client_id": os.getenv("FIREBASE_CLIENT_ID"),
    "auth_uri": os.getenv("FIREBASE_AUTH_URI"),
    "token_uri": os.getenv("FIREBASE_TOKEN_URI"),
    "auth_provider_x509_cert_url": os.getenv("FIREBASE_AUTH_PROVIDER_X509_CERT_URL"),
    "client_x509_cert_url": os.getenv("FIREBASE_CLIENT_X509_CERT_URL"),
    "universe_domain": os.getenv("FIREBASE_UNIVERSE_DOMAIN"),
}

In [3]:
cred = credentials.Certificate(service_account_info)
firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://coffee-shop-app-f9ec9.firebasestorage.app',
    'databaseURL': 'https://coffee-shop-app-f9ec9-default-rtdb.firebaseio.com/'
})

In [4]:
bucket = storage.bucket()

# Upload Data

In [5]:
image_folder_path = "./products/images/"

In [6]:
products_collection = db.reference("products")

In [7]:
df = pd.read_json("products/products.jsonl", lines=True)

In [8]:
df.head()

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,latte.jpg
2,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,chocolate_biscotti.jpg
3,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,espresso.jpg
4,Hazelnut Biscotti,Bakery,These delicious hazelnut biscotti are perfect ...,"[Flour, Sugar, Hazelnuts, Eggs, Baking Powder]",2.75,4.4,hazelnut_biscotti.jpg


In [9]:
def upload_image(bucket, image_path):
    image_name = image_path.split('/')[-1]
    blob = bucket.blob(f"product_images/{image_name}")
    # upload image
    blob.upload_from_filename(image_path)
    # make the image publicly accessible
    blob.make_public()
    return blob.public_url

In [10]:
for index, row in df.iterrows():
    print(index, row["name"])
    image_path = os.path.join(image_folder_path, row["image_path"])
    image_url = upload_image(bucket, image_path)
    print(image_url)
    product_data = row.to_dict()
    product_data.pop("image_path")
    product_data["image_url"] = image_url

    # add to firestore
    products_collection.push().set(product_data)

0 Cappuccino


NotFound: 404 POST https://storage.googleapis.com/upload/storage/v1/b/gs://coffee-shop-app-f9ec9.firebasestorage.app/o?uploadType=multipart: Not Found: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>)